In [1]:
import BioSimSpace as BSS

/home/li/miniconda3/envs/bss/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/li/miniconda3/envs/bss/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/li/miniconda3/envs/bss/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/li/miniconda3/envs/bss/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)



Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



/home/li/miniconda3/envs/bss/lib/python3.9/site-packages/BioSimSpace/Process/_process.py:611: UserWarning: The system contains a perturbable molecule .We will assume that you intend to simulate the lambda = 0 state. If you want to simulate the lambda = 1 state, then pass {'is_lambda1' : True} in the 'property_map' argument.
  _warnings.warn("The system contains a perturbable molecule ."
/home/li/miniconda3/envs/bss/lib/python3.9/site-packages/BioSimSpace/Process/_process.py:611: UserWarning: The system contains a perturbable molecule .We will assume that you intend to simulate the lambda = 0 state. If you want to simulate the lambda = 1 state, then pass {'is_lambda1' : True} in the 'property_map' argument.
  _warnings.warn("The system contains a perturbable molecule ."
/home/li/miniconda3/envs/bss/lib/python3.9/site-packages/BioSimSpace/Process/_process.py:611: UserWarning: The system contains a perturbable molecule .We will assume that you intend to simulate the lambda = 0 state. If y

In [2]:
ejm_46 = BSS.IO.readMolecules("org_protein_files/docked_ligands/ejm_46.sdf")[0]

In [3]:
ejm_50 = BSS.IO.readMolecules("org_protein_files/docked_ligands/ejm_50.sdf")[0]

In [4]:
#paramaterising protein - not working. done in tleap
AF_protein = BSS.IO.readMolecules(["org_protein_files/af_dry.rst7", "org_protein_files/af_dry.prm7"])

In [5]:
#paramaterising ligands
ejm_46 = BSS.Parameters.gaff2(ejm_46).getMolecule()
ejm_50 = BSS.Parameters.gaff2(ejm_50).getMolecule()

In [6]:
# mapping
mapping = BSS.Align.matchAtoms(ejm_46, ejm_50)

In [7]:
# inverse mapping
inv_mapping = {v: k for k, v in mapping.items()}

In [8]:
# Align ejm46 to ejm50 based on the inverse mapping.
ejm_50_aligned = BSS.Align.rmsdAlign(ejm_50, ejm_46, inv_mapping)

In [9]:
# Merge the ejm46 and ejm50 based on the mapping.
merged = BSS.Align.merge(ejm_46, ejm_50_aligned, mapping)

In [10]:
system = merged + AF_protein

In [11]:
# Check ligand dimensions
box_min, box_max = merged.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min,box_max)]
box_sizes = [x + 20 * BSS.Units.Length.angstrom for x in box_size]

In [12]:
# Creating box for free ligands
box, angles = BSS.Box.cubic(max(box_sizes))

In [13]:
# Solvation for unbounde state
solvated = BSS.Solvent.solvate('tip3p', molecule=merged, box=box, angles=angles, is_neutral=True)

# Check charge
solvated.charge()

4.7184e-16 |e|

In [14]:
# Check system dimensions
box_min, box_max = system.getAxisAlignedBoundingBox()
sys_box_size = [y - x for x, y in zip(box_min,box_max)]
sys_box_sizes = [x + 20 * BSS.Units.Length.angstrom for x in sys_box_size]

# Creating box for bounded system
box, angles = BSS.Box.cubic(max(sys_box_sizes))

# Solvation for bound state
system_solvated = BSS.Solvent.solvate('tip3p', molecule=system, box=box, angles=angles, is_neutral=True)

# Check charge
system_solvated.charge()

-1.5915e-07 |e|

In [15]:
# Save perturbable system to file
BSS.IO.savePerturbableSystem("pert_unbonded", solvated)
BSS.IO.savePerturbableSystem("pert_bonded", system_solvated)

In [16]:
# Define minimisation protocol
min_protocol = BSS.Protocol.Minimisation(steps=250)

In [17]:
# Minimisation - merged ligands
process = BSS.Process.Gromacs(system=solvated, protocol=min_protocol, work_dir="Minimisation/free_lig")
process.start()
process.wait()
minimised_ligands = process.getSystem()

In [18]:
# Minimisation - system
process = BSS.Process.Gromacs(system=system_solvated, protocol=min_protocol, work_dir="Minimisation/bound_sys").start()
process.wait()
minimised_system = process.getSystem()

In [19]:
# Equilibiration - free ligands - restrained NVT
eq_protocol = BSS.Protocol.Equilibration(runtime= 5*BSS.Units.Time.picosecond, temperature_start=0*BSS.Units.Temperature.kelvin, temperature_end=298*BSS.Units.Temperature.kelvin, restraint="all")
eq_process = BSS.Process.Gromacs(system=minimised_ligands, protocol=eq_protocol, work_dir="Equilibration/free_lig/res_NVT").start()
eq_process.wait()
eq_resNVT_unbound = eq_process.getSystem()

In [ ]:
# Equilibration - free ligands - unrestrained NVT
eq_protocol = BSS.Protocol.Equilibration(runtime= 50*BSS.Units.Time.picosecond, temperature=298*BSS.Units.Temperature.kelvin)
eq_process = BSS.Process.Gromacs(system=eq_resNVT_unbound, protocol=eq_protocol, work_dir="Equilibration/free_lig/unres_NVT").start()
eq_process.wait()
eq_unresNVT_unbound = eq_process.getSystem()

In [28]:
# Equilibration - free ligands - restrained NPT
eq_protocol = BSS.Protocol.Equilibration(runtime= 200*BSS.Units.Time.picosecond, temperature=298*BSS.Units.Temperature.kelvin, pressure=1*BSS.Units.Pressure.atm, restraint="heavy")
eq_process = BSS.Process.Gromacs(system=eq_unresNVT_unbound, protocol=eq_protocol, work_dir="Equilibration/free_lig/res_NPT")


In [23]:
# Equilibration - free ligands - restrained NPT
eq_protocol = BSS.Protocol.Equilibration(runtime= 200*BSS.Units.Time.picosecond, temperature=298*BSS.Units.Temperature.kelvin, pressure=1*BSS.Units.Pressure.atm, restraint="heavy")
eq_process = BSS.Process.Gromacs(system=eq_unresNVT_unbound, protocol=eq_protocol, work_dir="Equilibration/free_lig/res_NPT").start()
eq_process.wait()
eq_resNPT_unbound = eq_process.getSystem()

In [ ]:
# Equilibration - free ligands - restrained NPT
# gmx grompp -f gromacs.mdp -c ../unres_NVT/gromacs.gro -r ../unres_NVT/gromacs.gro -p gromacs.top -t ../unres_NVT/gromacs.cpt -o gromacs.tpr
# nohup gmx mdrun -deffnm gromacs -v > nohup_res_NPT.out &

In [34]:
# Equilibration - free ligands - unrestrained NPT
eq_protocol = BSS.Protocol.Equilibration(runtime= 200*BSS.Units.Time.picosecond, temperature=298*BSS.Units.Temperature.kelvin, pressure=1*BSS.Units.Pressure.atm)
eq_process = BSS.Process.Gromacs(system=eq_resNPT_unbound, protocol=eq_protocol, work_dir="Equilibration/free_lig/unres_NPT").start()
eq_process.wait()
eq_unbound = eq_process.getSystem()

In [36]:
# Equilibiration - system - restrained NVT
eq_protocol = BSS.Protocol.Equilibration(runtime= 5*BSS.Units.Time.picosecond, temperature_start=0*BSS.Units.Temperature.kelvin, temperature_end=298*BSS.Units.Temperature.kelvin, restraint="all")
eq_process = BSS.Process.Gromacs(system=minimised_system, protocol=eq_protocol, work_dir="Equilibration/bound_sys/res_NVT").start()
eq_process.wait()
eq_resNVT_system = eq_process.getSystem()

In [30]:
# Equilibiration - system - backbone restrained NVT
eq_protocol = BSS.Protocol.Equilibration(runtime= 50*BSS.Units.Time.picosecond, temperature=298*BSS.Units.Temperature.kelvin, restraint="backbone")
eq_process = BSS.Process.Gromacs(system=eq_resNVT_system, protocol=eq_protocol, work_dir="Equilibration/bound_sys/b_res_NVT").start()
eq_process.wait()
eq_bResNVT_system = eq_process.getSystem()

In [31]:
# Equilibiration - system - unrestrained NVT
eq_protocol = BSS.Protocol.Equilibration(runtime= 50*BSS.Units.Time.picosecond, temperature=298*BSS.Units.Temperature.kelvin)
eq_process = BSS.Process.Gromacs(system=eq_bResNVT_system, protocol=eq_protocol, work_dir="Equilibration/bound_sys/unres_NVT").start()
eq_process.wait()
eq_unresNVT_system = eq_process.getSystem()

In [32]:
# Equilibiration - system - restrained NPT
eq_protocol = BSS.Protocol.Equilibration(runtime= 200*BSS.Units.Time.picosecond, pressure=1*BSS.Units.Pressure.atm, temperature=298*BSS.Units.Temperature.kelvin, restraint="heavy")
eq_process = BSS.Process.Gromacs(system=eq_unresNVT_system, protocol=eq_protocol, work_dir="Equilibration/bound_sys/res_NPT").start()
eq_process.wait()
eq_resNPT_system = eq_process.getSystem()

In [33]:
# Equilibiration - system - unrestrained NPT
eq_protocol = BSS.Protocol.Equilibration(runtime= 200*BSS.Units.Time.picosecond, pressure=1*BSS.Units.Pressure.atm, temperature=298*BSS.Units.Temperature.kelvin)
eq_process = BSS.Process.Gromacs(system=eq_resNPT_system, protocol=eq_protocol, work_dir="Equilibration/bound_sys/unres_NPT").start()
eq_process.wait()
eq_system = eq_process.getSystem()

In [37]:
# Free energy protocol
FE_protocol = BSS.Protocol.FreeEnergy(timestep=2*BSS.Units.Time.femtosecond, runtime=4*BSS.Units.Time.nanosecond, num_lam=9)

In [38]:
# Config 2 legs
fep_unbound = BSS.FreeEnergy.Relative(eq_unbound, FE_protocol, work_dir="ejm46_ejm50/unbound")
fep_bound  = BSS.FreeEnergy.Relative(eq_system, FE_protocol, work_dir="ejm46_ejm50/bound")

In [40]:
# Run FEP
#fep_unbound.run()
#fep_bound.run()

ProcessRunner already started!


In [ ]:
# Analysis
pmf_unbound, overlap_unbound = fep_unbound.analyse()
pmf_bound,  overlap_bound  = fep_bound.analyse()

FE_binding = BSS.FreeEnergy.Relative.difference(pmf_bound, pmf_unbound)